In [1]:
!git clone https://github.com/Oneul-hyeon/Inquiry-Type-Classification.git
%cd Inquiry-Type-Classification
!git clone https://github.com/Oneul-hyeon/Modified-KoBERT.git
!pip install -r requirements.txt

Cloning into 'Inquiry-Type-Classification'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 9), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 727.36 KiB | 3.71 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/Inquiry-Type-Classification
Cloning into 'Modified-KoBERT'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 449 (delta 138), reused 120 (delta 110), pack-reused 273
Receiving objects: 100% (449/449), 227.95 KiB | 2.81 MiB/s, done.
Resolving deltas: 100% (226/226), done.
  Cloning https://github.com/ssut/py-hanspell.git (to revision fdc6ca50c19f1c85971437a072d89d4e5ce024b8) to /tmp/pip-install-w2_rynq0/py-hanspell_13fe6ec1dfc54fae87f7fc2742aea59f
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git

# 1.환경설정

## (1) 라이브러리 불러오기

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import joblib
import sys
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import nltk

sys.path.append('/content/Inquiry-Type-Classification/Modified-KoBERT')

from kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer
from kobert.utils import get_tokenizer

from sklearn.model_selection import train_test_split

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel
from sklearn.metrics import f1_score

device = torch.device('cuda')

## (2) 데이터 불러오기

In [3]:
data = joblib.load('/content/Inquiry-Type-Classification/preprocessed_dataset.pkl')

## (3) KoBERT 모델 불러오기

In [4]:
# KoBERT로부터 model, vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/Inquiry-Type-Classification/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/Inquiry-Type-Classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 2.Modeling

## (1) train/test dataset 분리

In [5]:
trainset, testset = train_test_split(data, test_size=0.1, shuffle=True, stratify= data['label'].values, random_state = 42)

## (2) KoBERT 모델의 입력 데이터 생성

In [6]:
# train_data 처리
dataset_train = []
for text, label in zip(trainset['text'], trainset['label'])  :
    data = []
    data.append(text)
    data.append(str(label))
    dataset_train.append(data)

# test_data 처리
dataset_test = []
for text, label in zip(testset['text'], testset['label'])  :
    data = []
    data.append(text)
    data.append(str(label))
    dataset_test.append(data)

## (3) 입력 데이터셋 토큰화

- 함수 정의

In [7]:
# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 함수 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

- Setting parameters

In [8]:
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 1000
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

- 토큰화

In [9]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model. /content/Inquiry-Type-Classification/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## (4) KoBERT 모델 학습

- EarlyStopping 함수 정의

In [10]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float("Inf")

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...")
        torch.save(model.state_dict(), "checkpoint.pt")
        self.val_loss_min = val_loss

- KoBERT 함수 정의

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [12]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# F1 스코어를 계산하는 함수 정의
def calc_f1_score(y_pred, y_true):
    y_pred = y_pred.detach().cpu().numpy()  # Tensor를 NumPy 배열로 변환합니다.
    y_pred = np.argmax(y_pred, axis=1)  # 예측값 중 가장 큰 값의 인덱스를 예측 클래스로 설정합니다.
    y_true = y_true.detach().cpu().numpy()  # GPU에서 CPU로 데이터를 옮깁니다.
    return f1_score(y_true, y_pred, average='weighted')

train_dataloader

- Training

In [13]:
train_history=[]
test_history=[]
loss_history=[]

early_stopping = EarlyStopping(patience=5, verbose=True) # EarlyStopping 적용

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_f1_score(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    y_pred, y_test = [], []
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_f1_score(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

    # Early stopping 및 ModelCheckpoint 적용
    epoch_test_loss = loss.item()
    early_stopping(epoch_test_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break

# 최적의 모델 불러오기
model.load_state_dict(torch.load("checkpoint.pt"))

  0%|          | 0/27 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.690071702003479 train acc 0.19769355492864735
epoch 1 train acc 0.173627559256988


  0%|          | 0/3 [00:00<?, ?it/s]

epoch 1 test acc 0.07604754432095415
Validation loss decreased (inf --> 1.525272). Saving model...


  0%|          | 0/27 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

epoch 46 batch id 1 loss 0.007560934405773878 train acc 1.0
epoch 46 train acc 0.995332757476961


  0%|          | 0/3 [00:00<?, ?it/s]

epoch 46 test acc 0.8257134245474026
EarlyStopping counter: 5 out of 5
Early stopping


<All keys matched successfully>

# 3. Model Save

In [14]:
torch.save(model, '/content/Inquiry-Type-Classification/model.pth')